### Problem 1

##Segmenting and Clustering Neighborhoods in Toronto

## Import libraries

In [51]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Scrape the List of postal codes of Canada

In [52]:
List_url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379"
source = requests.get(List_url).text

In [53]:
soup = BeautifulSoup(source, 'xml')

In [54]:
table=soup.find('table')

In [55]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [56]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [57]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Data Cleaning

remove rows where Borough is 'Not assigned'

In [58]:
# drop cells with a borough that is Not assigned
df=df[df['Borough']!='Not assigned']

Group neighborhoods in the same borough


(df['Neighborhood'] == 'Not assigned').describe()

In [59]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [60]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [61]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [62]:
df_merge.drop_duplicates(inplace=True)

In [63]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [64]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [65]:
df_merge.shape

(103, 3)

### Get Geo code

In [67]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data #geocoder didn't work so I downloaded the .csv file
geo = pd.read_csv("Geospatial_Coordinates.csv", index_col = 0)          #Read csv file
df_final = df.join(geo, on = "Postalcode")                                #Join dataframe 
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Heights,43.718518,-79.464763
6,M6A,North York,Lawrence Manor,43.718518,-79.464763
